In [ ]:
from SocialNetwork.social_network_environment import *
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
#from SocialNetwork.cascade import *
from Advertising.learners.Greedy_algorithm import *
from experiment_utilities import *

arms = arms_creation(seed = 30291, number_of_arms= 20)

exploration=4.2

deltas=[0.5,0.42,0.38,0.30,0.2,0.05]

social_network = social_network_environment()
social_network.init_from_json(json_path_network='Config/network.json', json_path_features='Config/features.json')

n_rounds=5000
vcg=VCG()
with open('Config/bids.json', 'r') as file:
    data = json.load(file)
    all_bids = data.get('bids')
bids=all_bids[0]

In [ ]:
all_rewards=[]
for arm in arms:
    print(arm)
    reward_total=0
    bids.insert(0, arm)
    ad_allocation_list = setup(bids=bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
    payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
    for j in range(1, n_rounds+1):
        active_nodes, total_reward = activate_cascade(social_network=social_network, 
                    ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (total_reward-payments_tot)
        reward_total += reward
        reset_nodes(social_network)
    reward_total=reward_total/n_rounds
    all_rewards.append(reward_total)
    bids.pop(0)
print(all_rewards)

In [ ]:
all_estimations_f=[]
for i in range(0, len(all_bids)):
    all_rewards=[]
    bids=all_bids[i]
    for arm in arms:
        reward_total=0
        bids.insert(0, arm)
        ad_allocation_list = setup(bids=bids, n_bids=5)
        best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
        for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        for j in range(1, n_rounds+1):
            active_nodes, total_reward = activate_cascade(social_network=social_network, 
                        ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
            payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
            reward = (total_reward-payments_tot)
            reward_total += reward
            reset_nodes(social_network)
        reward_total=reward_total/n_rounds
        all_rewards.append(reward_total)
        bids.pop(0)
    all_estimations_f.append(max(all_rewards))

In [ ]:
print(all_estimations_f)